## CAPSTONE PROJECT DATA PREPARATION (STATS GENERATION AND PLAYER ROLES CLEANING)
## AUTHOR: Simon Lee
## DATE CREATED: JUNE 30th 2022
## LAST DATE ADJUSTED: AUG 14th 2022

Downloading Packages

In [250]:
import pandas as pd
import csv as csv
import os as os
import numpy as np

### IMPORTING ALL THE DATA NECESSARY

In [251]:

ball = pd.read_csv(r"/Users/simonlee/Desktop/Capstone/Ball by Ball Data.csv")
player = pd.read_csv(r"/Users/simonlee/Desktop/Capstone/Data/people.csv")
roles = pd.read_csv(r"/Users/simonlee/Desktop/Capstone/Data/roles.csv")
lineups = pd.read_csv(r"/Users/simonlee/Desktop/Capstone/Data/team_lineups.csv")

#Dropping unneccessary column
##roles.drop('Unnamed: 0', axis = 1, inplace = True)


In [295]:
rem = ball[ball["Match ID"].str.contains("International")]
Int_teams = ['india', 'west indies', 'england', 'australia', 'new zealand', 'pakistan', 'bangladesh', 'afghanistan', 'south africa']
y = []
for i, r in rem.iterrows():
    if r["Batting Team"].lower() in Int_teams:
        y.append(r["Match ID"])

In [296]:
len(list(set(y)))

819

In [253]:
ball.head(2)

,Match ID,Ball,Batting Team,Striker,Striker ID,Non_Striker,Non_Striker ID,Bowler,Bowler ID,Runs,Extras,Wicket,How_out,Batsman Out
0,Big Bash897705.csv,0.1,Perth Scorchers,M Klinger,b970a03f,SE Marsh,508a1ea7,MG Neser,0164b064,0,0,0,NaN,NaN
1,Big Bash897705.csv,0.2,Perth Scorchers,M Klinger,b970a03f,SE Marsh,508a1ea7,MG Neser,0164b064,0,0,0,NaN,NaN


## CHANGING DATA TYPES WHERE NECESSARY

In [254]:
roles['PlayerID'] = roles['PlayerID'].apply(str)
player['identifier'] = player['identifier'].apply(str)
player.dropna(axis = 0, subset = ['key_cricinfo'], inplace = True)
player['key_cricinfo'] = player['key_cricinfo'].astype(int).astype(str)
ball["Striker ID"] = ball["Striker ID"].apply(str)
ball["Bowler ID"] = ball["Bowler ID"].apply(str)

In [255]:
for col in lineups.columns:
    lineups[col] = lineups[col].apply(str)

### Differnciate Wickets attributed to Blowlers vs not

In [256]:
b_wickets = []

for w in ball['How_out']:
    if w in ['caught', 'bowled', 'stumped', 'lbw','caught and bowled', 'retired hurt', 'hit wicket','handled the ball']:
        b_wickets.append(1)
    else:
        b_wickets.append(0)
ball['Bowlers Wicket'] = b_wickets


In [257]:
# TO GET BATTING STATISTICS

bat_stats = ball.groupby(["Striker ID"]).agg({"Bowler ID": "count", "Runs":"sum", "Wicket": "sum"}).reset_index().rename(columns = {"Bowler ID": "Number of Balls Faced", "Wicket": "Dismissals"})
bat_stats["Strike Rate"] = round(bat_stats["Runs"]/bat_stats["Number of Balls Faced"]*100, 2)
bat_stats["Average"] = round(bat_stats["Runs"]/bat_stats["Dismissals"],2)
bat_stats.sort_values("Number of Balls Faced", ascending = False).head(2)

,Striker ID,Number of Balls Faced,Runs,Dismissals,Strike Rate,Average
3032,db584dad,9439,12914,363,136.82,35.58
2600,ba607b88,7643,9831,254,128.63,38.70


In [258]:
def avg_clean_up(a):
    if a["Dismissals"] == 0:
        return a["Runs"]
    else:
        return a["Average"]



In [259]:
bat_stats["Average"] = bat_stats.apply(avg_clean_up, axis = 1)

In [260]:
# TO GET BOWLING STATISTICS

bowl_stats = ball.groupby(["Bowler ID"]).agg({"Striker ID": "count", "Runs":"sum", "Bowlers Wicket": "sum"}).reset_index().rename(columns = {"Striker ID": "Number of Balls Bowled", "Bowlers Wicket": "Wickets"})
bowl_stats["Strike Rate"] = round(bowl_stats["Runs"]/bowl_stats["Wickets"], 2)
bowl_stats["Economy"] = round(bowl_stats["Runs"]/bowl_stats["Number of Balls Bowled"] * 6,2)
bowl_stats.sort_values("Wickets", ascending = False).head(2)

,Bowler ID,Number of Balls Bowled,Runs,Wickets,Strike Rate,Economy
1410,87e562a9,9425,11873,517,22.97,7.56
981,5f547c8b,6949,7013,410,17.10,6.06


### TO REMOVE ALL PEOPLE WITH INF AS THEIR STRIKE RATE

In [261]:
def sr_clean_up(a):
    if a["Wickets"] == 0:
        return a["Runs"]
    else:
        return a["Strike Rate"]

bowl_stats["Strike Rate"] = bowl_stats.apply(sr_clean_up, axis = 1)

### TO GET PLAYER DETAILS IN ONE DF

In [262]:

df1 = player.drop(['key_bigbash', 'key_cricbuzz', 'key_crichq', 'key_cricinfo_2', 'key_cricingif', 'key_cricketarchive', 'key_cricketarchive_2', 'key_opta', 'key_opta_2', 'key_pulse', 'key_pulse_2'], axis = 1)
combined = df1.merge(roles, how = 'left', left_on='key_cricinfo', right_on='PlayerID')
player_roles = combined[combined['PlayerID'].notna()].drop('PlayerID', axis = 1)
player_roles.head(3)

,identifier,name,unique_name,key_cricinfo,Full Name,Born,Age,Batting Style,Bowling Style,Playing Role
0,b4a23876,AAA Amsterdam,AAA Amsterdam,772407,Alex Adrian Anthony Amsterdam,"June 08, 1991, Berbice, Guyana",31y 36d,Left hand Bat,Right arm Offbreak,Top order Batter
2,fd3c5c00,AAA Patel,AAA Patel,1163136,Ayaj Ashraf Ali Patel,"May 28, 1996",26y 48d,Right hand Bat,NaN,NaN
4,02d11602,AA Banner,AA Banner,590021,Andrew Alexander Banner,"July 04, 1998",24y 10d,Right hand Bat,Right arm Medium,NaN


## GETTING THE DETAILS OF PLAYERS INVOLVED IN A BALL (STRIKER AND BOWLER TYPES)

In [263]:
df2= ball.merge(player_roles, how = 'left', left_on = 'Bowler ID', right_on = 'identifier').drop([ 'Full Name', 'Born', 
'Age', 'Batting Style', 'Playing Role', 'identifier', 'name', 'unique_name', 'key_cricinfo'], axis = 1).rename(columns = {'Bowling Style': 'Bowler Type'})

detailed_ball = df2.merge(player_roles, how = 'left', left_on = 'Striker ID', right_on = 'identifier').drop([ 'Full Name', 'Born', 
'Age', 'Playing Role', 'identifier', 'name', 'unique_name', 'key_cricinfo', 'Bowling Style'], axis = 1)

detailed_ball['Bowler Type'] = detailed_ball['Bowler Type'].fillna("Unknown")




### CATEGORIZING BOWLING TYPES

In [264]:
detailed_ball['Bowler Type'].unique()

array(['Right arm Medium fast', 'Slow Left arm Orthodox',
       'Right arm Fast medium', 'Right arm Offbreak', 'Right Legbreak',
       'Left arm Fast medium', 'Right arm Fast', 'Right arm Medium',
       'Left arm Wrist spin', 'Right Legbreak Googly',
       'Right arm Medium, Right arm Offbreak', 'Left arm Fast',
       'Left arm Medium, Slow Left arm Orthodox, Left arm Wrist spin',
       'Left arm Medium fast', 'Left Legbreak',
       'Right arm Offbreak, Legbreak', 'Left arm Medium',
       'Right arm Medium fast, Legbreak', 'Left Legbreak Googly',
       'Unknown', 'Right arm Medium fast, Right arm Offbreak',
       'Right arm Medium, Legbreak',
       'Left arm Fast medium, Slow Left arm Orthodox',
       'Right arm Offbreak, Legbreak Googly',
       'Right arm Fast medium, Legbreak',
       'Right arm Offbreak, Slow Left arm Orthodox', 'Right arm Bowler',
       'Right arm Medium, Right arm Slow',
       'Right arm Medium, Legbreak Googly', 'Left arm Slow',
       'Right arm F

In [266]:
lst = []
for bowl_type in list(detailed_ball['Bowler Type']):
    if "right" in str(bowl_type).lower():
        if "fast" in str(bowl_type).lower():
            lst.append('Right Arm Fast')
        elif "medium" in str(bowl_type).lower():
            lst.append("Right Arm Fast")
        elif "slow" in str(bowl_type).lower():
            lst.append("Right Arm Spinner")
        elif "legbreak" in str(bowl_type).lower():
            lst.append("Right Arm Spinner")
        elif "offbreak" in str(bowl_type).lower():
            lst.append("Right Arm Spinner")
        else:
            lst.append("Right Arm Fast")
    elif "left" in str(bowl_type).lower():
        if "fast" in str(bowl_type).lower():
            lst.append('Left Arm Fast')
        elif "medium" in str(bowl_type).lower():
            lst.append("Left Arm Fast")
        elif "slow" in str(bowl_type).lower():
            lst.append("Left Arm Spinner")
        elif "legbreak" in str(bowl_type).lower():
            lst.append("Left Arm Spinner")
        elif "offbreak" in str(bowl_type).lower():
            lst.append("Left Arm Spinner")
        else:
            lst.append(bowl_type)
    else:
        lst.append(bowl_type)

detailed_ball['Bowling Category'] = lst

## GETTING BATTINNG STATS AGAINST EACH BOWLING CATEGORY

In [267]:
for b_cat in detailed_ball['Bowling Category'].unique():
    bcat = b_cat.replace(" ", "")
    globals()[f"temp_{bcat}"] = detailed_ball[(detailed_ball['Bowling Category'] == b_cat)]
    globals()[f"temp_{bcat}"] = globals()[f"temp_{bcat}"].groupby(["Striker", "Striker ID"]).agg({"Bowler ID": "count", "Runs":"sum", "Bowlers Wicket": "sum"}).reset_index().rename(columns = {"Bowler ID": ("Number of Balls Faced Vs " + b_cat), "Bowlers Wicket": ("Dismissals Vs " + b_cat), "Runs": ("Runs Vs " + b_cat)})
    globals()[f"temp_{bcat}"][("Strike Rate Vs " + b_cat)] = round(globals()[f"temp_{bcat}"][("Runs Vs " + b_cat )]/globals()[f"temp_{bcat}"][("Number of Balls Faced Vs " + b_cat)]*100, 2)
    globals()[f"temp_{bcat}"][("Average Vs " + b_cat)] = round(globals()[f"temp_{bcat}"][("Runs Vs " + b_cat )]/globals()[f"temp_{bcat}"][("Dismissals Vs " + b_cat)],2)



### MERGING ALL STATS INTO ONE DF

In [268]:
all_bat_stats = bat_stats
for b_cat in detailed_ball['Bowling Category'].unique():
    bcat = b_cat.replace(" ", "")
    all_bat_stats = all_bat_stats.merge(globals()[f"temp_{bcat}"], how = 'left', left_on= 'Striker ID', right_on= 'Striker ID').drop(['Striker'], axis = 1)

#Filling in all blanks with 0's
for i in all_bat_stats.columns:
    all_bat_stats[i] = all_bat_stats[i].fillna(0)

### CLEANING UP AGERAGES OF PEOPLE WHO DID NOT GET OUT TO A PARTICULAR CATEGORY OF BOWLER

* IF RUNS VS THAT CATEGORY OF BOWLER > OVERALL AVERAGE THEN RUNS VS THAT TYPE OF BOWLER ELSE OVERALL AVERAGE

In [269]:

for cat in detailed_ball['Bowling Category'].unique():
    x = []
    for i, r in all_bat_stats.iterrows():
        if r["Dismissals Vs " + cat] == 0:
            if r["Runs Vs " + cat] > r["Average"]:
                x.append(r["Runs Vs " + cat])
            else:
                x.append( r["Average"])
        else:
            x.append(r["Average Vs " + cat])
    all_bat_stats ["Average Vs " + cat]  = x

In [270]:
all_bat_stats[all_bat_stats["Average Vs Left Arm Fast"] == np.inf]

,Striker ID,Number of Balls Faced,Runs,Dismissals,Strike Rate,Average,Number of Balls Faced Vs Right Arm Fast,Runs Vs Right Arm Fast,Dismissals Vs Right Arm Fast,Strike Rate Vs Right Arm Fast,...,Number of Balls Faced Vs Left arm Wrist spin,Runs Vs Left arm Wrist spin,Dismissals Vs Left arm Wrist spin,Strike Rate Vs Left arm Wrist spin,Average Vs Left arm Wrist spin,Number of Balls Faced Vs Unknown,Runs Vs Unknown,Dismissals Vs Unknown,Strike Rate Vs Unknown,Average Vs Unknown


In [271]:
all_bat_stats.to_csv(r"/Users/simonlee/Desktop/Capstone/Data/Batting Stats.csv")

### GETTING ALL THE PLAYERS AND INFERRING PLAYING ROLES FOR THOSE WHO DO NOT HAVE USING USAGE OF THE PLAYER

In [272]:
strikers = list(ball["Striker ID"].unique())
bowlers = list(ball["Bowler ID"].unique())
a = strikers + bowlers
all_players = list(set(a))
ap = pd.DataFrame(data=all_players, columns = ['ID'])

In [273]:
len(all_players)

3853

In [274]:
all = ap.merge(bat_stats, how = 'left', left_on='ID', right_on='Striker ID').merge(bowl_stats, how = 'left', left_on = 'ID', right_on = 'Bowler ID').rename(
    columns = {"Runs_x": "Batting Runs", "Runs_y": "Runs off Bowling" ,"Strike Rate_x": "Batting Strike Rate", "Strike Rate_y": "Bowling Strike Rate", "Average_x": "Batting Average", "Average_y": "Bowling Average"  })
all = all.drop(columns=['Striker ID', 'Bowler ID'], axis = 1)
for i in all.columns:
    all[i] = all[i].fillna(0)

all.head(10)

,ID,Number of Balls Faced,Batting Runs,Dismissals,Batting Strike Rate,Average,Number of Balls Bowled,Runs off Bowling,Wickets,Bowling Strike Rate,Economy
0,1cfac535,309.0,325.0,28.0,105.18,11.61,1842.0,2431.0,96.0,25.32,7.92
1,c2988809,21.0,18.0,2.0,85.71,9.00,0.0,0.0,0.0,0.00,0.00
2,5390f178,1.0,1.0,0.0,100.00,1.00,48.0,48.0,3.0,16.00,6.00
3,0c1da0c4,99.0,83.0,3.0,83.84,27.67,0.0,0.0,0.0,0.00,0.00
4,1558d83b,69.0,63.0,9.0,91.30,7.00,1248.0,1526.0,60.0,25.43,7.34
5,d86f6c69,589.0,714.0,24.0,121.22,29.75,0.0,0.0,0.0,0.00,0.00
6,214cf493,7.0,5.0,0.0,71.43,5.00,49.0,73.0,3.0,24.33,8.94
7,c16d4035,2688.0,3550.0,147.0,132.07,24.15,0.0,0.0,0.0,0.00,0.00
8,c2233c17,3.0,0.0,1.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00
9,4bf5c12f,8.0,10.0,1.0,125.00,10.00,12.0,19.0,1.0,19.00,9.50


In [ ]:
num_match = []
for id in all["ID"]:
    num_match.append(len(set(ball[(ball["Striker ID"] == id) | (ball["Bowler ID"] == id)]["Match ID"])))
all["Number of Matches Played"] = num_match


In [ ]:
all["Balls Bowled Per Match"] = all["Number of Balls Bowled"]/all["Number of Matches Played"]
all["Balls Faced Per Match"] = all["Number of Balls Faced"]/all["Number of Matches Played"]


In [303]:
all["Batting Index"] = (all["Average"]*all["Batting Strike Rate"])/50

In [304]:
all.to_csv(r"/Users/simonlee/Desktop/Capstone/Data/Player Stats Generated.csv", index = 0)

In [ ]:


x = []
for i, r in all.iterrows():
    if r["Balls Bowled Per Match"] == 0:
        x.append("Batter")
    elif r["Balls Faced Per Match"] == 0:
        x.append("Bowler")
    elif r["Balls Bowled Per Match"] >= 6 and r["Balls Faced Per Match"] <= 10:
        x.append( "Bowler")
    elif r["Balls Bowled Per Match"] <= 6 and r["Balls Faced Per Match"] >= 10:
        x.append("Batter")
    elif r["Balls Bowled Per Match"] >= 6 and r["Balls Faced Per Match"] >= 10:
        x.append("Allrounder")
    else:
        if r["Balls Bowled Per Match"] > r["Balls Faced Per Match"]:
            x.append("Bowler")
        else:
            x.append("Batter")
all["Player Role Inferred"] = x

In [ ]:
df1 = player.drop(['key_bigbash', 'key_cricbuzz', 'key_crichq', 'key_cricinfo_2', 'key_cricingif', 'key_cricketarchive', 'key_cricketarchive_2', 'key_opta', 'key_opta_2', 'key_pulse', 'key_pulse_2'], axis = 1)

temp = all.merge(df1, how = 'left', left_on = 'ID', right_on='identifier').drop(['identifier', 'name', 'unique_name'], axis = 1)

all_needed = temp.drop(['Number of Balls Faced', 'Batting Runs', 'Dismissals',
       'Batting Strike Rate', 'Average', 'Number of Balls Bowled',
       'Runs off Bowling', 'Wickets', 'Bowling Strike Rate', 'Economy',
       'Number of Matches Played', 'Balls Bowled Per Match',
       'Balls Faced Per Match'], axis = 1)

all_needed


,ID,Player Role Inferred,key_cricinfo
0,1cfac535,Bowler,310525
1,c2988809,Batter,1094350
2,5390f178,Bowler,774211
3,0c1da0c4,Batter,1163174
4,1558d83b,Bowler,499660
...,...,...,...
4454,aeab0d4f,Bowler,339095
4455,e8915c11,Bowler,303874
4456,fea7a34c,Bowler,404458
4457,7dde58c9,Batter,1046603


In [ ]:
rc1 = roles.merge(all_needed, how = 'left', left_on = 'PlayerID', right_on = 'key_cricinfo')
rc1[rc1['Playing Role'].isna()]

,PlayerID,Full Name,Born,Age,Batting Style,Bowling Style,Playing Role,ID,Player Role Inferred,key_cricinfo
1,596426,Stefan Tait,"March 14, 1996",26y 123d,Left hand Bat,Left arm Fast,NaN,02635912,Bowler,596426
4,460176,Gregory Irving,"July 14, 1973, Jamaica",49y,Left hand Bat,Left arm Fast,NaN,17530fd9,Bowler,460176
7,334621,Shadley Claude van Schalkwyk,"August 05, 1988, Cape Town, Cape Province",33y 343d,Left hand Bat,Left arm Fast,NaN,97b0e4ff,Bowler,334621
13,543041,Mbulelo Budaza,"September 06, 1993, Grahamstown, eastern cape",28y 311d,Right hand Bat,Left arm Fast,NaN,1d105638,Bowler,543041
17,595954,Duan Jansen,"May 01, 2000",22y 74d,Right hand Bat,Left arm Fast,NaN,8dc152d1,Bowler,595954
...,...,...,...,...,...,...,...,...,...,...
4452,1163180,Gareth Johnson,"February 03, 1986",36y 161d,NaN,NaN,NaN,ceec82cd,Allrounder,1163180
4453,1046639,Muhammad Armghan Khan,"November 22, 1980",41y 234d,NaN,NaN,NaN,cf785dd1,Batter,1046639
4454,1197880,Zeko Burgess,NaN,NaN,NaN,NaN,NaN,d0042e16,Bowler,1197880
4455,446972,NaN,NaN,NaN,NaN,NaN,NaN,d01f8cb9,Bowler,446972


In [297]:
## Categoriving Bowling Types

lst = []
for bowl_type in list(rc1['Bowling Style']):
    if "right" in str(bowl_type).lower():
        if "fast" in str(bowl_type).lower():
            lst.append('Right Arm Fast')
        elif "medium" in str(bowl_type).lower():
            lst.append("Right Arm Fast")
        elif "slow" in str(bowl_type).lower():
            lst.append("Right Arm Spinner")
        elif "legbreak" in str(bowl_type).lower():
            lst.append("Right Arm Spinner")
        elif "offbreak" in str(bowl_type).lower():
            lst.append("Right Arm Spinner")
        else:
            lst.append("Right Arm Fast")
    elif "left" in str(bowl_type).lower():
        if "fast" in str(bowl_type).lower():
            lst.append('Left Arm Fast')
        elif "medium" in str(bowl_type).lower():
            lst.append("Left Arm Fast")
        elif "slow" in str(bowl_type).lower():
            lst.append("Left Arm Spinner")
        elif "legbreak" in str(bowl_type).lower():
            lst.append("Left Arm Wrist Spinner")
        elif "offbreak" in str(bowl_type).lower():
            lst.append("Left Arm Spinner")
        elif "wrist" in str(bowl_type).lower():
            lst.append("Left Arm Spinner")
        else:
            lst.append(bowl_type)
    else:
        lst.append("NoBowl")

rc1['Bowling Style Cats'] = lst

In [286]:
bats = []

for i, r in rc1.iterrows():
    if pd.isnull(r["Batting Style"]):
        bats.append("NoBat")
    else:
        bats.append(r["Batting Style"])

rc1["Batting Style"] = bats

In [298]:
temp1 = []
for i, r in rc1.iterrows():
    if pd.isnull(r['Playing Role']):
        temp1.append(r["Player Role Inferred"])
    else:
        if "wicket" in str(r["Playing Role"]).lower():
            temp1.append("Wicketkeeper")
        elif str(r["Playing Role"]).lower() == "batsman":
            temp1.append("Batter")
        else:
            temp1.append(r["Playing Role"])
        

rc1['Player Role Cleaned'] = temp1
rc1.to_csv(r"/Users/simonlee/Desktop/Capstone/Data/roles1.csv")

In [288]:
temp1

['Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowling Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowling Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Batter',
 'Bowling Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowling Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowling Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Allrounder',
 'Allrounder',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Batter',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
 'Bowler',
